In [3]:
import pandas as pd
import torch
from sklearn.utils import shuffle

In [4]:
import pandas as pd
rnames = ['userId', 'movieId', 'rating', "TimeStamp"]
ratings = pd.read_table("ml-1m.inter", header=0, names=rnames, engine='python')
ratings

,userId,movieId,rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
ratio = 0.8
ratings = shuffle(ratings)
# ratings
train = ratings.copy()
test = ratings.copy()
train.iloc[int(ratio*len(ratings)):,2] = None
# train.iloc[int(ratio*len(ratings)):,'rating'] = 0
test.iloc[:int(ratio*len(ratings)),2] = None

In [6]:
# test

In [7]:
# train

In [8]:
ratings.describe()

,userId,movieId,rating,TimeStamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [9]:
rating_matrix = train.pivot(index='userId', columns='movieId', values='rating')
n_users, n_movies = rating_matrix.shape
# Scaling ratings to between 0 and 1, this helps our model by constraining predictions
min_rating, max_rating = train['rating'].min(), train['rating'].max()
rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)
print(rating_matrix)
print(n_users*n_movies-rating_matrix.isnull().values.sum())

movieId  1     2     3     4     5     6     7     8     9     10    ...   
userId                                                               ...   
1         1.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  \
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   NaN   NaN   NaN  0.25   NaN   NaN   NaN   NaN  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
6036      NaN   NaN   NaN  0.25   NaN  0.50   NaN   NaN   NaN   NaN  ...   
6037      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6038      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6039      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6040      0.5   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieId  39

In [10]:
# Replacing missing ratings with -1 so we can filter them out later

rating_matrix[rating_matrix.isnull()] = -1
rating_matrix = torch.FloatTensor(rating_matrix.values)
print(rating_matrix)

tensor([[ 1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        ...,
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [ 0.5000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]])


In [11]:
# # This is how we can define our feature matrices
# # We're going to be training these, so we'll need gradients
# latent_vectors = 5
# user_features = torch.randn(n_users, latent_vectors, requires_grad=True)
# user_features.data.mul_(0.01)
# movie_features = torch.randn(n_movies, latent_vectors, requires_grad=True)
# movie_features.data.mul_(0.01)

In [12]:
class FCFLoss(torch.nn.Module):
    def __init__(self, lam_u=0.3, lam_v=0.3):
        super().__init__()
        self.lam_u = lam_u
        self.lam_v = lam_v
    
    def forward(self, matrix, u_features, v_features):
        non_zero_mask = (matrix != -1).type(torch.FloatTensor)
        predicted = torch.sigmoid(torch.mm(u_features, v_features.t()))
        
        diff = (matrix - predicted)**2
        prediction_error = torch.sum(diff*non_zero_mask)

        u_regularization = self.lam_u * torch.sum(u_features.norm(dim=1))
        v_regularization = self.lam_v * torch.sum(v_features.norm(dim=1))
        
        return prediction_error + u_regularization + v_regularization, prediction_error

In [22]:
# FCF

num_client = 200
m = n_users//num_client
num_epoch = 100   
latent_vectors = 20
# user_features = torch.randn(n_users, latent_vectors, requires_grad=True)
# user_features.data.mul_(0.01)

user_features = []
for i in range(num_client):
    user_features.append(torch.randn(m, latent_vectors, requires_grad=True))
# user_features = [torch.randn(m, latent_vectors, requires_grad=True)]*num_client
movie_features = torch.randn(n_movies, latent_vectors, requires_grad=True)
movie_features.data.mul_(0.01)

fcferror = FCFLoss(lam_u=0.1, lam_v=0.1)
# optimizer_local = torch.optim.Adam([user_features], lr=0.01)

optimizer_client = []

# print(m)
for i in range(num_client):
    optimizer = torch.optim.Adam([user_features[i]], lr=0.01)
    optimizer_client.append(optimizer)

error_list = []
previous = torch.zeros(n_movies, latent_vectors)
optimizer_server = torch.optim.Adam([movie_features], lr=0.01)
# print(optimizer_set)
for step, epoch in enumerate(range(num_epoch)):
    optimizer_server.zero_grad()
    # local update
    total_loss = 0
    aver_prediction_error = 0
    for i in range(num_client):
        optimizer_client[i].zero_grad()
        loss, prediction_error = fcferror(rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)
        aver_prediction_error += prediction_error/num_client
        total_loss += loss
        # print(movie_features[0])
        loss.backward()
        optimizer_client[i].step()
        
    # server update
    optimizer_server.step()

    # stop criterion
    error = (torch.norm(movie_features-previous,p=2))/torch.norm(previous,p=2)
    # print(error.detach().numpy())
    error_list.append(error.detach().numpy())
    previous = movie_features.clone()

    if step % 10 == 0:
        print(f"Step {step}, {aver_prediction_error:.3f}")
        # print(f"Step {step}, {total_loss:.3f}")

Step 0, 394.600
Step 10, 266.328
Step 20, 226.289
Step 30, 212.838
Step 40, 203.559
Step 50, 195.230
Step 60, 188.050
Step 70, 181.897
Step 80, 176.514
Step 90, 171.816


In [23]:
test_rating_matrix = test.pivot(index='userId', columns='movieId', values='rating')

test_rating_matrix[test_rating_matrix.isnull()] = -1
test_rating_matrix = torch.FloatTensor(test_rating_matrix.values)
# test_rating_matrix

In [24]:
print(movie_features)

tensor([[ 0.0914, -0.0783,  0.0097,  ..., -0.1366,  0.0301,  0.1973],
        [ 0.0440, -0.0231, -0.0384,  ..., -0.0274, -0.0301,  0.0326],
        [-0.0029,  0.0014, -0.0384,  ...,  0.0424, -0.0445,  0.0004],
        ...,
        [ 0.0015, -0.0044,  0.0047,  ..., -0.0059, -0.0045,  0.0043],
        [ 0.0042, -0.0033,  0.0046,  ..., -0.0042, -0.0061,  0.0042],
        [ 0.0309, -0.0526,  0.0257,  ..., -0.0257, -0.0772,  0.0521]],
       requires_grad=True)


In [25]:
non_zero_mask = (test_rating_matrix != -1).type(torch.FloatTensor)
num = torch.sum(non_zero_mask)
def Error(matrix, u_features, v_features):
    predicted_ratings = torch.sigmoid(torch.mm(u_features,v_features.t()))
    # predicted_ratings = non_zero_mask * predicted_ratings
    pred = (predicted_ratings*(max_rating - min_rating) + min_rating)*non_zero_mask[i*m:(i+1)*m]
    actual = matrix*non_zero_mask[i*m:(i+1)*m]
    # print(pred)
    # print(actual)
    AE_diff = torch.abs(pred - actual)
    SE_diff = (pred - actual)**2
    
    prediction_abs_error = torch.sum(AE_diff)
    prediction_squared_error = torch.sum(SE_diff)
    return prediction_abs_error, prediction_squared_error

AE_error = 0
SE_error = 0

for i in range(num_client):
    AE_error += Error(test_rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)[0]
    SE_error += Error(test_rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)[1]
test_MAE = AE_error/num
test_RMSE = torch.sqrt(SE_error/num)
print('test_MAE =', test_MAE.data.numpy())
print('test_RMSE =', test_RMSE.data.numpy())

test_MAE = 0.70305955
test_RMSE = 0.889433


In [26]:
import numpy as np
def topk_metrics(y_true, y_pred, topKs=[10]):
	"""choice topk metrics and compute it
	the metrics contains 'ndcg', 'mrr', 'recall' and 'hit'

	Args:
		y_true: list, 2-dim, each row contains the items that the user interacted
		y_pred: list, 2-dim, each row contains the items recommended  
		topKs: list or tuple, if you want to get top5 and top10, topKs=(5, 10)

	Return:
		results: list, it contains five metrics, 'ndcg', 'recall', 'mrr', 'hit', 'precision'

	"""
	assert len(y_true) == len(y_pred)

	if not isinstance(topKs, (tuple, list)):
		raise ValueError('topKs wrong, it should be tuple or list')

	ndcg_result = []
	mrr_result = []
	hit_result = []
	precision_result = []
	recall_result = []
	for idx in range(len(topKs)):
		ndcgs = 0
		mrrs = 0
		hits = 0
		precisions = 0
		recalls = 0
		for i in range(len(y_true)):
			if len(y_true[i]) != 0:
				mrr_tmp = 0
				mrr_flag = True
				hit_tmp = 0
				dcg_tmp = 0
				idcg_tmp = 0
				hit = 0
				for j in range(topKs[idx]):
					if y_pred[i][j] in y_true[i]:
						hit += 1.
						if mrr_flag:
							mrr_flag = False
							mrr_tmp = 1. / (1 + j)
							hit_tmp = 1.
						dcg_tmp += 1. / (np.log2(j + 2))
					idcg_tmp += 1. / (np.log2(j + 2))
				hits += hit_tmp
				mrrs += mrr_tmp
				recalls += hit / len(y_true[i])
				precisions += hit / topKs[idx]
				if idcg_tmp != 0:
					ndcgs += dcg_tmp / idcg_tmp
		hit_result.append(round(hits / len(y_pred), 4))
		mrr_result.append(round(mrrs / len(y_pred), 4))
		recall_result.append(round(recalls / len(y_pred), 4))
		precision_result.append(round(precisions / len(y_pred), 4))
		ndcg_result.append(round(ndcgs / len(y_pred), 4))

	results = []
	for idx in range(len(topKs)):
		results.append(hit_result[idx])
	return results

In [27]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

non_zero_mask = (test_rating_matrix != -1).type(torch.FloatTensor)
num = torch.sum(non_zero_mask)
print(num)
def recommend(u_features, v_features, mask, k):
    scores = (torch.sigmoid(torch.mm(u_features,v_features.t()))).data.numpy()
    # scores = (torch.sigmoid(torch.mm(u_features,v_features.t()))).data.numpy()
    recommend_list = []
    # print(scores)
    for i in range(len(scores)):
    # print(torch.sum(mask))
    # print(scores.shape)
        score = scores[i]
        sorted_id = sorted(range(len(score)), key=lambda k: score[k], reverse=True)[:k]
        recommend_list.append(sorted_id)
    return recommend_list
def ground_truth(actual):
    actual_list = []
    for i in range(len(actual)):
        actual_id = [j for j, e in enumerate(actual[i]) if e != 0]
        actual_list.append(actual_id)
    return actual_list


In [28]:
Hit_Ratio_10 = 0
for i in range(num_client):
    actual_matrix = (test_rating_matrix[i*m:(i+1)*m]*non_zero_mask[i*m:(i+1)*m]).data.numpy()
    # print(actual_matrix)
    actual = ground_truth(actual_matrix)
    predict = recommend(user_features[i], movie_features,non_zero_mask[i*m:(i+1)*m],10)
    results = topk_metrics(actual, predict,topKs=(5,10))
    # print(results)
    Hit_Ratio_10 += results[1]/num_client
    # print(i)
print(f'HR@{10}: {Hit_Ratio_10}')

In [29]:
def score_f(u_features, v_features, mask, k):
    scores = (torch.sigmoid(torch.mm(u_features,v_features.t()))).data.numpy()
    # scores = (torch.sigmoid(torch.mm(u_features,v_features.t()))).data.numpy()
    recommend_list = []
    score_list = []
    # print(scores)
    for i in range(len(scores)):
    # print(torch.sum(mask))
    # print(scores.shape)
        score = scores[i]
        sorted_id = sorted(range(len(score)), key=lambda k: score[k], reverse=True)[:k]
        sorted_score = [score[j] for j in sorted_id]
        recommend_list.append(sorted_id)
        score_list.append(sorted_score)
    return recommend_list, score_list
def rel(actual_matrix, recommend):
    rel_list = []
    for i in range(len(actual_matrix)):
        actual_vector = actual_matrix[i]
        rel_id = [actual_vector[j] for j in recommend[i]]
        rel_list.append(rel_id)
    return rel_list

In [30]:
import numpy as np
from sklearn import metrics

ndcg_10 = 0
ndcg_20 = 0
for i in range(num_client):
# for i in range(1):
    actual_matrix = (test_rating_matrix[i*m:(i+1)*m]*non_zero_mask[i*m:(i+1)*m]).data.numpy()
    # print(actual_matrix)
    recommend, score = score_f(user_features[i], movie_features,non_zero_mask[i*m:(i+1)*m],10)
    true_relevance = rel(actual_matrix, recommend)

    ndcg_10 += metrics.ndcg_score(true_relevance, score, k=10)/num_client
    # ndcg_20 += metrics.ndcg_score(true_relevance, score, k=20)/num_client
print(f'NDCG@{10}: {ndcg_10}')
# print(f'NDCG@{20}: {ndcg_20}')